In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import json

In [2]:
heroes_info = pd.read_csv('csv/heroes_done.csv').set_index('Unnamed: 0')

In [3]:
train = pd.read_csv('csv/train.csv').set_index('id')
test = pd.read_csv('csv/test.csv').set_index('id')

train['istrain'] = 1
test['istrain'] = 0

train = pd.concat([train, test], axis=0, sort=True)

In [4]:
heroes_train = pd.DataFrame([heroes_info.loc[hero] for hero in train['hero_id']])

In [5]:
heroes_train = heroes_train.set_index(train.index)

In [6]:
train = pd.concat([train, heroes_train], axis=1)

In [7]:
train['is_player_win'] = train['player_team'] == train['winner_team']
train = train.drop(['player_team', 'winner_team'], axis=1)

In [8]:
train['duration_in_mins'] = train['duration'] / 60

In [23]:
train['gold_lose'] = train['gold_earned'] - train['gold_spent']
train['gold_lose_ratio'] = train['gold_lose'] / train['gold_earned']

In [10]:
train['strange_magic_param'] = (train['kills'] + train['assists'] + train['denies']) / train['duration']

In [11]:
for feature in ['hero_damage', 'last_hits', 'stuns']:
    train[feature + '_per_min'] = train[feature] / train['duration_in_mins']

In [12]:
train['avg_kda_x16'] = (train['avg_kills_x16'] + train['avg_assists_x16']) / train['avg_deaths_x16']
train['kda'] = (train['kills'] + train['assists']) / train['deaths']

In [13]:
for par in ['str', 'agi', 'int']:
    train[par] = train['base_' + par] + train[par + '_gain'] + train['level']

In [14]:
for par in [('health', 'str', 20), ('health_regen', 'str', 0.1), ('armor', 'agi', 0.16), ('mana', 'int', 12), ('mana_regen', 'int', 0.05)]:
    train[par[0]] = train['base_' + par[0]] + train[par[1]] * par[2]

# Json Lines

In [15]:
def mmmm(table, result):
    for feature in table.keys():
        result[feature + '_mean'] = np.mean(table[feature])
        result[feature + '_median'] = np.median(table[feature])
        result[feature + '_max'] = np.max(table[feature])
        result[feature + '_min'] = np.min(table[feature])

In [16]:
def kill_log(record):
    damage_targets = record['damage_targets']
    result = {}
    sorted_targets = {}
    for target in damage_targets.keys():
        kind = target.split('_')[2]
        if kind in ['observer', 'sentry']:
            result[kind + '_ward_damage'] = damage_targets[target]
        else:
            if kind in ['neutral', 'hero']:
                pass
            elif kind in ['creep']:
                if target.split('_')[3] == 'goodguys':
                    kind = 'friendly_creeps' if record['player_team'] == 'dire' else 'enemy_creeps'
                else:
                    kind = 'friendly_creeps' if record['player_team'] == 'radiant' else 'enemy_creeps'
            else:
                kind = 'hero_minions'
            kind += '_damage'
            targets = sorted_targets.get(kind, [])
            targets.append(damage_targets[target])
            sorted_targets[kind] = targets

    mmmm(sorted_targets, result)
    
    return result

In [17]:
def gold_log(record):
            
    series = record['series']

    drop_count = sum([1 for s in list(map(list, zip(record['series']['time'], record['series']['player_gold'], record['series']['dire_gold'], record['series']['radiant_gold']))) if 0 in s or s[0] < 0])
    for p in series.keys(): series[p] = series[p][drop_count:]

    radiant = 'friendly_gold' if record['player_team'] == 'radiant' else 'enemy_gold'
    dire = 'friendly_gold' if record['player_team'] == 'dire' else 'enemy_gold'

    series[radiant] = series.pop('radiant_gold')
    series[dire] = series.pop('dire_gold')

    for p in ['player_gold', 'friendly_gold', 'enemy_gold']:
        for t in range(len(series['time']) - 1):
            arr = series.get(p + '_growth', [])
            arr.append(series[p][t + 1] - series[p][t])
            series[p + '_growth'] = arr
        series[p + '_growth'] += [np.mean(series[p + '_growth'])]

    for p in ['friendly_gold', 'enemy_gold']:
        series[p + '_ratio'] = np.array(series['player_gold']) / np.array(series[p])
        series[p + '_growth_ratio'] = np.array(series['player_gold_growth']) / np.array(series[p + '_growth'])

    series['teams_gold_ratio'] = np.array(series['friendly_gold']) / np.array(series['enemy_gold'])

    result = {}

    result['gold_earned'] = series['player_gold'][-1]
    result['gold_after_warmup'] = series['player_gold'][0]

    series = {k: series[k] for k in list(series.keys())[4:]}

    mmmm(series, result)
    
    return result

In [18]:
def other_heroes(record):
    friendly_heroes = record['radiant_heroes'] if record['player_team'] == 'radiant' else record['dire_heroes']
    enemy_heroes = record['radiant_heroes'] if record['player_team'] != 'radiant' else record['dire_heroes']

    hero_ids = list(heroes_info.index)

    result = {}

    for hero_id in hero_ids:
        if hero_id in friendly_heroes:
            result[hero_id] = 1
        elif hero_id in enemy_heroes:
            result[hero_id] = -1
        else:
            result[hero_id] = 0
    return result

In [19]:
def level_up_times(record):
    level_up_times = record['level_up_times']
    level_up_times_diff = {'level_up_times': [level_up_times[p + 1] - level_up_times[p] for p in range(len(level_up_times) - 1)]} if len(level_up_times) > 1 else {'level_up_times': [0]}
    result = {}
    mmmm(level_up_times_diff, result)
    return result

In [20]:
damage_targets, teams_heroes, serieses, level_times = {}, {}, {}, {}

with open('jsonlines/train.jsonlines') as file:
    for i, line in enumerate(file):
        record = json.loads(line)
        damage_targets[record['id']] = kill_log(record)
        teams_heroes[record['id']] = other_heroes(record)
        serieses[record['id']] = gold_log(record)
        level_times[record['id']] = level_up_times(record)
        if i % 10 == 0:
            print('\r{}'.format(i), end='')

with open('jsonlines/test.jsonlines') as file:
    for i, line in enumerate(file):
        record = json.loads(line)
        damage_targets[record['id']] = kill_log(record)
        teams_heroes[record['id']] = other_heroes(record)
        serieses[record['id']] = gold_log(record)
        level_times[record['id']] = level_up_times(record)
        if i % 10 == 0:
            print('\r{}'.format(i), end='')

43260

In [21]:
serieses = pd.DataFrame(serieses).T
teams_heroes = pd.DataFrame(teams_heroes).T
damage_targets = pd.DataFrame(damage_targets).T
level_times = pd.DataFrame(level_times).T

In [22]:
train = pd.concat([train, serieses, teams_heroes, damage_targets, level_times], axis=1)

In [24]:
train.to_csv('my_train.csv')

### Добавляем мусор

In [84]:
train = pd.read_csv('my_train.csv')

In [101]:
abilities = pd.read_csv('trash/abilities_final.csv')
items_count = pd.read_csv('trash/items_count_final.csv')
final_items = pd.read_csv('trash/final_items.csv')

In [102]:
abilities = abilities.drop(['istrain', 'Unnamed: 0'], axis=1).sort_index()
items_count = items_count.drop(['istrain', 'Unnamed: 0'], axis=1).sort_index()
final_items = final_items.drop(['Unnamed: 0'], axis=1).sort_index()

In [105]:
train = pd.concat([train, abilities, items_count, final_items], axis=1)

In [106]:
train = train.fillna(0)

In [107]:
X_train = train.loc[train['istrain'] == 1]
y_train = X_train['skilled']
X_train = X_train.drop(['istrain', 'skilled'], axis=1)

X_test = train.loc[train['istrain'] == 0]
X_test = X_test.drop(['istrain', 'skilled'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [108]:
import catboost
clf = catboost.CatBoostClassifier(depth=8, iterations=5500)
clf.fit(X_train, y_train)

Learning rate set to 0.013499
0:	learn: 0.6900159	total: 316ms	remaining: 28m 56s
1:	learn: 0.6867578	total: 622ms	remaining: 28m 30s
2:	learn: 0.6837087	total: 912ms	remaining: 27m 51s
3:	learn: 0.6807096	total: 1.23s	remaining: 28m 5s
4:	learn: 0.6774626	total: 1.55s	remaining: 28m 29s
5:	learn: 0.6745872	total: 1.83s	remaining: 28m
6:	learn: 0.6716331	total: 2.14s	remaining: 27m 58s
7:	learn: 0.6689137	total: 2.42s	remaining: 27m 45s
8:	learn: 0.6661972	total: 2.73s	remaining: 27m 48s
9:	learn: 0.6637177	total: 3.03s	remaining: 27m 43s
10:	learn: 0.6611410	total: 3.33s	remaining: 27m 39s
11:	learn: 0.6586493	total: 3.63s	remaining: 27m 38s
12:	learn: 0.6561283	total: 3.92s	remaining: 27m 33s
13:	learn: 0.6537402	total: 4.21s	remaining: 27m 31s
14:	learn: 0.6514000	total: 4.51s	remaining: 27m 27s
15:	learn: 0.6491567	total: 4.79s	remaining: 27m 23s
16:	learn: 0.6468489	total: 5.1s	remaining: 27m 25s
17:	learn: 0.6447265	total: 5.39s	remaining: 27m 22s
18:	learn: 0.6425569	total: 5.71

155:	learn: 0.5203879	total: 48.4s	remaining: 27m 36s
156:	learn: 0.5199351	total: 48.6s	remaining: 27m 35s
157:	learn: 0.5193550	total: 48.9s	remaining: 27m 33s
158:	learn: 0.5189451	total: 49.2s	remaining: 27m 33s
159:	learn: 0.5185151	total: 49.5s	remaining: 27m 31s
160:	learn: 0.5182000	total: 49.7s	remaining: 27m 29s
161:	learn: 0.5177750	total: 50s	remaining: 27m 28s
162:	learn: 0.5173896	total: 50.3s	remaining: 27m 27s
163:	learn: 0.5170408	total: 50.6s	remaining: 27m 25s
164:	learn: 0.5166139	total: 50.9s	remaining: 27m 25s
165:	learn: 0.5162038	total: 51.2s	remaining: 27m 23s
166:	learn: 0.5158415	total: 51.5s	remaining: 27m 23s
167:	learn: 0.5153855	total: 51.8s	remaining: 27m 23s
168:	learn: 0.5150467	total: 52s	remaining: 27m 21s
169:	learn: 0.5146228	total: 52.3s	remaining: 27m 20s
170:	learn: 0.5143157	total: 52.6s	remaining: 27m 20s
171:	learn: 0.5140107	total: 53.1s	remaining: 27m 23s
172:	learn: 0.5136587	total: 53.4s	remaining: 27m 24s
173:	learn: 0.5131837	total: 53.

306:	learn: 0.4777899	total: 1m 33s	remaining: 26m 15s
307:	learn: 0.4776087	total: 1m 33s	remaining: 26m 13s
308:	learn: 0.4774418	total: 1m 33s	remaining: 26m 12s
309:	learn: 0.4772247	total: 1m 33s	remaining: 26m 11s
310:	learn: 0.4769332	total: 1m 34s	remaining: 26m 11s
311:	learn: 0.4767226	total: 1m 34s	remaining: 26m 9s
312:	learn: 0.4765701	total: 1m 34s	remaining: 26m 8s
313:	learn: 0.4763070	total: 1m 34s	remaining: 26m 8s
314:	learn: 0.4761437	total: 1m 35s	remaining: 26m 6s
315:	learn: 0.4760028	total: 1m 35s	remaining: 26m 5s
316:	learn: 0.4758437	total: 1m 35s	remaining: 26m 4s
317:	learn: 0.4755443	total: 1m 35s	remaining: 26m 3s
318:	learn: 0.4753067	total: 1m 36s	remaining: 26m 2s
319:	learn: 0.4751551	total: 1m 36s	remaining: 26m
320:	learn: 0.4750031	total: 1m 36s	remaining: 25m 58s
321:	learn: 0.4748113	total: 1m 36s	remaining: 25m 58s
322:	learn: 0.4745757	total: 1m 37s	remaining: 25m 57s
323:	learn: 0.4744067	total: 1m 37s	remaining: 25m 56s
324:	learn: 0.4742330	

457:	learn: 0.4534977	total: 2m 11s	remaining: 24m 4s
458:	learn: 0.4533812	total: 2m 11s	remaining: 24m 3s
459:	learn: 0.4532838	total: 2m 11s	remaining: 24m 2s
460:	learn: 0.4531696	total: 2m 11s	remaining: 24m 1s
461:	learn: 0.4530080	total: 2m 12s	remaining: 24m 1s
462:	learn: 0.4528757	total: 2m 12s	remaining: 24m
463:	learn: 0.4527605	total: 2m 12s	remaining: 23m 59s
464:	learn: 0.4526558	total: 2m 12s	remaining: 23m 59s
465:	learn: 0.4525310	total: 2m 13s	remaining: 23m 58s
466:	learn: 0.4523886	total: 2m 13s	remaining: 23m 57s
467:	learn: 0.4522803	total: 2m 13s	remaining: 23m 57s
468:	learn: 0.4521537	total: 2m 13s	remaining: 23m 56s
469:	learn: 0.4520481	total: 2m 14s	remaining: 23m 55s
470:	learn: 0.4519509	total: 2m 14s	remaining: 23m 54s
471:	learn: 0.4518369	total: 2m 14s	remaining: 23m 54s
472:	learn: 0.4516982	total: 2m 14s	remaining: 23m 53s
473:	learn: 0.4515296	total: 2m 15s	remaining: 23m 53s
474:	learn: 0.4513313	total: 2m 15s	remaining: 23m 52s
475:	learn: 0.45119

607:	learn: 0.4363193	total: 2m 47s	remaining: 22m 30s
608:	learn: 0.4362409	total: 2m 48s	remaining: 22m 29s
609:	learn: 0.4361588	total: 2m 48s	remaining: 22m 28s
610:	learn: 0.4360833	total: 2m 48s	remaining: 22m 28s
611:	learn: 0.4359985	total: 2m 48s	remaining: 22m 27s
612:	learn: 0.4359175	total: 2m 48s	remaining: 22m 27s
613:	learn: 0.4358235	total: 2m 49s	remaining: 22m 26s
614:	learn: 0.4357229	total: 2m 49s	remaining: 22m 25s
615:	learn: 0.4356522	total: 2m 49s	remaining: 22m 25s
616:	learn: 0.4355715	total: 2m 49s	remaining: 22m 24s
617:	learn: 0.4354784	total: 2m 50s	remaining: 22m 23s
618:	learn: 0.4353517	total: 2m 50s	remaining: 22m 23s
619:	learn: 0.4352673	total: 2m 50s	remaining: 22m 22s
620:	learn: 0.4351610	total: 2m 50s	remaining: 22m 22s
621:	learn: 0.4350320	total: 2m 51s	remaining: 22m 22s
622:	learn: 0.4349590	total: 2m 51s	remaining: 22m 21s
623:	learn: 0.4348863	total: 2m 51s	remaining: 22m 20s
624:	learn: 0.4348001	total: 2m 51s	remaining: 22m 19s
625:	learn

...

4894:	learn: 0.2728758	total: 20m 15s	remaining: 2m 30s
4895:	learn: 0.2728417	total: 20m 16s	remaining: 2m 30s
4896:	learn: 0.2728232	total: 20m 16s	remaining: 2m 29s
4897:	learn: 0.2728009	total: 20m 16s	remaining: 2m 29s
4898:	learn: 0.2727789	total: 20m 16s	remaining: 2m 29s
4899:	learn: 0.2727531	total: 20m 17s	remaining: 2m 29s
4900:	learn: 0.2727336	total: 20m 17s	remaining: 2m 28s
4901:	learn: 0.2727134	total: 20m 17s	remaining: 2m 28s
4902:	learn: 0.2726848	total: 20m 17s	remaining: 2m 28s
4903:	learn: 0.2726685	total: 20m 18s	remaining: 2m 28s
4904:	learn: 0.2726492	total: 20m 18s	remaining: 2m 27s
4905:	learn: 0.2726239	total: 20m 18s	remaining: 2m 27s
4906:	learn: 0.2726037	total: 20m 18s	remaining: 2m 27s
4907:	learn: 0.2725790	total: 20m 18s	remaining: 2m 27s
4908:	learn: 0.2725585	total: 20m 19s	remaining: 2m 26s
4909:	learn: 0.2725371	total: 20m 19s	remaining: 2m 26s
4910:	learn: 0.2725139	total: 20m 19s	remaining: 2m 26s
4911:	learn: 0.2724918	total: 20m 19s	remaining:

5042:	learn: 0.2698012	total: 20m 52s	remaining: 1m 53s
5043:	learn: 0.2697815	total: 20m 52s	remaining: 1m 53s
5044:	learn: 0.2697586	total: 20m 52s	remaining: 1m 52s
5045:	learn: 0.2697266	total: 20m 52s	remaining: 1m 52s
5046:	learn: 0.2697017	total: 20m 53s	remaining: 1m 52s
5047:	learn: 0.2696800	total: 20m 53s	remaining: 1m 52s
5048:	learn: 0.2696605	total: 20m 53s	remaining: 1m 51s
5049:	learn: 0.2696395	total: 20m 53s	remaining: 1m 51s
5050:	learn: 0.2696124	total: 20m 54s	remaining: 1m 51s
5051:	learn: 0.2695940	total: 20m 54s	remaining: 1m 51s
5052:	learn: 0.2695742	total: 20m 54s	remaining: 1m 50s
5053:	learn: 0.2695533	total: 20m 54s	remaining: 1m 50s
5054:	learn: 0.2695273	total: 20m 55s	remaining: 1m 50s
5055:	learn: 0.2694976	total: 20m 55s	remaining: 1m 50s
5056:	learn: 0.2694763	total: 20m 55s	remaining: 1m 49s
5057:	learn: 0.2694609	total: 20m 55s	remaining: 1m 49s
5058:	learn: 0.2694392	total: 20m 56s	remaining: 1m 49s
5059:	learn: 0.2694165	total: 20m 56s	remaining:

5190:	learn: 0.2668689	total: 21m 28s	remaining: 1m 16s
5191:	learn: 0.2668485	total: 21m 28s	remaining: 1m 16s
5192:	learn: 0.2668258	total: 21m 28s	remaining: 1m 16s
5193:	learn: 0.2668076	total: 21m 29s	remaining: 1m 15s
5194:	learn: 0.2667878	total: 21m 29s	remaining: 1m 15s
5195:	learn: 0.2667689	total: 21m 29s	remaining: 1m 15s
5196:	learn: 0.2667530	total: 21m 29s	remaining: 1m 15s
5197:	learn: 0.2667313	total: 21m 29s	remaining: 1m 14s
5198:	learn: 0.2667109	total: 21m 30s	remaining: 1m 14s
5199:	learn: 0.2666895	total: 21m 30s	remaining: 1m 14s
5200:	learn: 0.2666573	total: 21m 30s	remaining: 1m 14s
5201:	learn: 0.2666388	total: 21m 30s	remaining: 1m 13s
5202:	learn: 0.2666247	total: 21m 31s	remaining: 1m 13s
5203:	learn: 0.2666033	total: 21m 31s	remaining: 1m 13s
5204:	learn: 0.2665801	total: 21m 31s	remaining: 1m 13s
5205:	learn: 0.2665650	total: 21m 31s	remaining: 1m 12s
5206:	learn: 0.2665416	total: 21m 32s	remaining: 1m 12s
5207:	learn: 0.2665222	total: 21m 32s	remaining:

5340:	learn: 0.2639359	total: 22m 4s	remaining: 39.4s
5341:	learn: 0.2639214	total: 22m 5s	remaining: 39.2s
5342:	learn: 0.2639082	total: 22m 5s	remaining: 38.9s
5343:	learn: 0.2638918	total: 22m 5s	remaining: 38.7s
5344:	learn: 0.2638709	total: 22m 5s	remaining: 38.4s
5345:	learn: 0.2638437	total: 22m 6s	remaining: 38.2s
5346:	learn: 0.2638267	total: 22m 6s	remaining: 38s
5347:	learn: 0.2638027	total: 22m 6s	remaining: 37.7s
5348:	learn: 0.2637922	total: 22m 6s	remaining: 37.5s
5349:	learn: 0.2637728	total: 22m 7s	remaining: 37.2s
5350:	learn: 0.2637521	total: 22m 7s	remaining: 37s
5351:	learn: 0.2637373	total: 22m 7s	remaining: 36.7s
5352:	learn: 0.2637265	total: 22m 7s	remaining: 36.5s
5353:	learn: 0.2637023	total: 22m 7s	remaining: 36.2s
5354:	learn: 0.2636808	total: 22m 8s	remaining: 36s
5355:	learn: 0.2636619	total: 22m 8s	remaining: 35.7s
5356:	learn: 0.2636342	total: 22m 8s	remaining: 35.5s
5357:	learn: 0.2636155	total: 22m 9s	remaining: 35.2s
5358:	learn: 0.2635973	total: 22m 

5490:	learn: 0.2610202	total: 22m 42s	remaining: 2.23s
5491:	learn: 0.2610035	total: 22m 43s	remaining: 1.99s
5492:	learn: 0.2609889	total: 22m 43s	remaining: 1.74s
5493:	learn: 0.2609661	total: 22m 43s	remaining: 1.49s
5494:	learn: 0.2609452	total: 22m 43s	remaining: 1.24s
5495:	learn: 0.2609284	total: 22m 43s	remaining: 993ms
5496:	learn: 0.2609082	total: 22m 44s	remaining: 745ms
5497:	learn: 0.2608902	total: 22m 44s	remaining: 496ms
5498:	learn: 0.2608763	total: 22m 44s	remaining: 248ms
5499:	learn: 0.2608589	total: 22m 44s	remaining: 0us


In [ ]:
get_feature_importance

In [112]:
result = pd.DataFrame(list(map(int, clf.predict(X_test))))
result.columns = ['skilled']
result.index = X_test.id
result.to_csv('new_way1.csv')
result

,skilled
id,
0,1
1,1
4,0
5,1
9,0
10,0
11,0
17,1
24,0


In [ ]:
from sklearn.metrics import accuracy_score
print('Validation accuracy:', accuracy_score(y_test, clf1.predict(X_test)))

In [ ]:
pd.DataFrame(clf1.predict(X_test)).to_csv('0.824868013835791.csv')